In [1]:
# Load libraries
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from scipy import stats
import seaborn as sns
import numpy as np

In [2]:
import requests # library to handle requests
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [3]:
df = pd.read_csv("hawker_centre_modified_short.csv")
#Adding the dataset and turning it into a dataframe
df.head()

,name_of_centre,postal_code,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls,Latitude,Longitude
0,Adam Road Food Centre,289876,"2, Adam Road",HC,Government,32,32,0,1.32926,103.81452
1,Amoy Street Food Centre,69111,"National Development Building, Annex B, Telok ...",HC,Government,135,134,1,1.28944,103.84998
2,Bedok Food Centre,469572,"1, Bedok Road",HC,Government,32,32,0,1.32659,103.95121
3,Beo Crescent Market,169982,"38A, Beo Crescent",MHC,Government,94,32,62,1.28864,103.82764
4,Berseh Food Centre,208877,"166, Jalan Besar",HC,Government,66,66,0,1.30578,103.85554


In [4]:
df.describe()

,postal_code,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls,Latitude,Longitude
count,33.000000,33.000000,33.000000,33.00000,33.000000,33.000000
mean,288985.272727,116.545455,58.545455,58.00000,1.310383,103.842926
std,174100.126842,84.297127,31.542720,72.50862,0.026512,0.046327
min,48940.000000,21.000000,16.000000,0.00000,1.280730,103.721626
25%,151115.000000,63.000000,32.000000,0.00000,1.289440,103.817009
50%,208877.000000,94.000000,51.000000,48.00000,1.305780,103.839730
75%,424768.000000,161.000000,83.000000,82.00000,1.320440,103.865840
max,618499.000000,365.000000,134.000000,302.00000,1.380500,103.951210


# Now I use the foursquare api

In [5]:
CLIENT_ID = 'TD5QDZZRT1HRKRDRY3ELQIJI4DNBT3X3IAKLLETWN4B1OYBN' # your Foursquare ID
CLIENT_SECRET = 'Q3H2QHFFJ5NYLYN5R1S2TG0NS4JR2IHY3BCCWFORPV534JR5' # your Foursquare Secret
VERSION = '20200420'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TD5QDZZRT1HRKRDRY3ELQIJI4DNBT3X3IAKLLETWN4B1OYBN
CLIENT_SECRET:Q3H2QHFFJ5NYLYN5R1S2TG0NS4JR2IHY3BCCWFORPV534JR5


In [6]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['CentreName', 
                  'Centre Latitude', 
                  'Centre Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
dp = getNearbyVenues(names=df['name_of_centre'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Adam Road Food Centre
Amoy Street Food Centre
Bedok Food Centre
Beo Crescent Market
Berseh Food Centre
Bukit Timah Market
Chomp Chomp Food Centre
Commonwealth Crescent Market
Dunman Food Centre
East Coast Lagoon Food Village
Geylang Serai Market
Golden Mile Food Centre
Holland Village Market & Food Centre
Kallang Estate Market
Market Street Food Centre
Newton Food Centre
North Bridge Road Market & Food Centre
Pasir Panjang Food Centre
Serangoon Garden Market
Sembawang Hill Food Centre
Taman Jurong Market & Food Centre
Tanglin Halt Market
Tiong Bahru Market
Zion Riverside Food Centre
ABC Brickworks Market & Food Centre
Albert Centre Market & Food Centre
Alexandra Village Food Centre
Blk 1 Jalan Kukoh
Blk 105 Hougang Ave 1
Blk 11 Telok Blangah Crescent
Blk 112 Jalan Bukit Merah
Blk 115 Bukit Merah View
Blk 117 Aljunied Ave 2


In [8]:
dp.head()

,CentreName,Centre Latitude,Centre Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adam Road Food Centre,1.32926,103.81452,Akane Restaurant,1.330851,103.815565,Japanese Restaurant
1,Adam Road Food Centre,1.32926,103.81452,Hollandse Club,1.329966,103.816648,Hotel
2,Adam Road Food Centre,1.32926,103.81452,Donguri,1.331121,103.815887,Japanese Restaurant
3,Adam Road Food Centre,1.32926,103.81452,7Adam,1.329441,103.813910,Modern European Restaurant
4,Adam Road Food Centre,1.32926,103.81452,Lounge,1.331060,103.815864,Lounge


In [9]:
dp.groupby('Venue Category').count()

,CentreName,Centre Latitude,Centre Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,2,2,2,2,2,2
Accessories Store,3,3,3,3,3,3
American Restaurant,2,2,2,2,2,2
Arcade,1,1,1,1,1,1
Art Gallery,7,7,7,7,7,7
...,...,...,...,...,...,...
Wine Bar,2,2,2,2,2,2
Wine Shop,5,5,5,5,5,5
Wings Joint,2,2,2,2,2,2


In [10]:
dp.shape

(1417, 7)

In [11]:
# one hot encoding
dp_onehot = pd.get_dummies(dp[['Venue Category']], prefix="", prefix_sep="")

# add CentreName column back to dataframe
dp_onehot['CentreName'] = dp['CentreName'] 

# move CentreName column to the first column
fixed_columns = [dp_onehot.columns[-1]] + list(dp_onehot.columns[:-1])
dp_onehot = dp_onehot[fixed_columns]

dp_onehot.head()

,CentreName,ATM,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,...,Video Store,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adam Road Food Centre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Adam Road Food Centre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Adam Road Food Centre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Adam Road Food Centre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adam Road Food Centre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
dp_onehot.shape

(1417, 192)

In [13]:
dp_grouped = dp_onehot.groupby('CentreName').mean().reset_index()
dp_grouped

,CentreName,ATM,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,...,Video Store,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,ABC Brickworks Market & Food Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Adam Road Food Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Albert Centre Market & Food Centre,0.000000,0.017241,0.000000,0.000000,0.034483,0.017241,0.017241,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Alexandra Village Food Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040816,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Amoy Street Food Centre,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000,0.00,0.000000,...,0.000000,0.013158,0.000000,0.013158,0.000000,0.000000,0.013158,0.000000,0.000000,0.039474
5,Bedok Food Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Beo Crescent Market,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Berseh Food Centre,0.000000,0.011364,0.000000,0.000000,0.000000,0.000000,0.034091,0.00,0.000000,...,0.000000,0.011364,0.000000,0.000000,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000
8,Blk 1 Jalan Kukoh,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,0.00,0.016949,...,0.000000,0.000000,0.000000,0.000000,0.016949,0.016949,0.000000,0.000000,0.000000,0.016949
9,Blk 105 Hougang Ave 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.00,0.000000,...,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
dp_grouped.shape

(33, 192)

In [15]:
num_top_venues = 5

for cent in dp_grouped['CentreName']:
    print("----"+cent+"----")
    temp = dp_grouped[dp_grouped['CentreName'] == cent].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ABC Brickworks Market & Food Centre----
                  venue  freq
0    Chinese Restaurant  0.21
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.08
3            Food Court  0.08
4      Asian Restaurant  0.08


----Adam Road Food Centre----
                        venue  freq
0         Japanese Restaurant  0.29
1                       Hotel  0.14
2  Modern European Restaurant  0.14
3                        Café  0.14
4                      Lounge  0.14


----Albert Centre Market & Food Centre----
                venue  freq
0  Chinese Restaurant  0.07
1        Dessert Shop  0.07
2                Café  0.07
3               Hotel  0.05
4         Art Gallery  0.03


----Alexandra Village Food Centre----
                venue  freq
0  Chinese Restaurant  0.12
1         Coffee Shop  0.10
2   Indian Restaurant  0.04
3          Food Court  0.04
4         Supermarket  0.04


----Amoy Street Food Centre----
                 venue  freq
0  Japanese Restaurant  0.05
1          Yoga

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['CentreName']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
centres_venues_sorted = pd.DataFrame(columns=columns)
centres_venues_sorted['CentreName'] = dp_grouped['CentreName']

for ind in np.arange(dp_grouped.shape[0]):
    centres_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dp_grouped.iloc[ind, :], num_top_venues)

centres_venues_sorted.head()

,CentreName,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABC Brickworks Market & Food Centre,Chinese Restaurant,Food Court,Coffee Shop,Fast Food Restaurant,Asian Restaurant,Café,Bistro,Supermarket,Malay Restaurant,Noodle House
1,Adam Road Food Centre,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
2,Albert Centre Market & Food Centre,Dessert Shop,Café,Chinese Restaurant,Hotel,Japanese Restaurant,Shopping Mall,Art Gallery,Sushi Restaurant,Ice Cream Shop,Bakery
3,Alexandra Village Food Centre,Chinese Restaurant,Coffee Shop,Indian Restaurant,Juice Bar,Food Court,Bakery,Sporting Goods Shop,Café,Supermarket,Asian Restaurant
4,Amoy Street Food Centre,Japanese Restaurant,Yoga Studio,Nightclub,Italian Restaurant,Cocktail Bar,Hotel,Bar,Bistro,Shopping Mall,History Museum


In [18]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

dp_grouped_clustering = dp_grouped.drop('CentreName', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dp_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 2, 0, 2, 2, 1, 0, 2, 0])

In [19]:
# add clustering labels
centres_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

centre_merged = dp

# merge toronto_grouped with toronto_data to add latitude/longitude for each hawker centre
centre_merged = centre_merged.join(centres_venues_sorted.set_index('CentreName'), on='CentreName')

centre_merged.head() # check the last columns!

,CentreName,Centre Latitude,Centre Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adam Road Food Centre,1.32926,103.81452,Akane Restaurant,1.330851,103.815565,Japanese Restaurant,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,Adam Road Food Centre,1.32926,103.81452,Hollandse Club,1.329966,103.816648,Hotel,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
2,Adam Road Food Centre,1.32926,103.81452,Donguri,1.331121,103.815887,Japanese Restaurant,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
3,Adam Road Food Centre,1.32926,103.81452,7Adam,1.329441,103.813910,Modern European Restaurant,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
4,Adam Road Food Centre,1.32926,103.81452,Lounge,1.331060,103.815864,Lounge,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop


In [23]:
centre_merged.loc[centre_merged['Cluster Labels'] == 0, centre_merged.columns[[1] + list(range(5, centre_merged.shape[1]))]]

,Centre Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
122,1.30578,103.858309,Whisky Bar,0,Indian Restaurant,Chinese Restaurant,Hotel,Café,Vegetarian / Vegan Restaurant,Hostel,Asian Restaurant,Food Court,Bakery,Dessert Shop
123,1.30578,103.854911,Hostel,0,Indian Restaurant,Chinese Restaurant,Hotel,Café,Vegetarian / Vegan Restaurant,Hostel,Asian Restaurant,Food Court,Bakery,Dessert Shop
124,1.30578,103.854063,Hostel,0,Indian Restaurant,Chinese Restaurant,Hotel,Café,Vegetarian / Vegan Restaurant,Hostel,Asian Restaurant,Food Court,Bakery,Dessert Shop
125,1.30578,103.856986,Dim Sum Restaurant,0,Indian Restaurant,Chinese Restaurant,Hotel,Café,Vegetarian / Vegan Restaurant,Hostel,Asian Restaurant,Food Court,Bakery,Dessert Shop
126,1.30578,103.853602,Indian Restaurant,0,Indian Restaurant,Chinese Restaurant,Hotel,Café,Vegetarian / Vegan Restaurant,Hostel,Asian Restaurant,Food Court,Bakery,Dessert Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,1.32044,103.887853,Asian Restaurant,0,Food Court,Asian Restaurant,Chinese Restaurant,Grocery Store,Vegetarian / Vegan Restaurant,Noodle House,Fast Food Restaurant,Bistro,Bakery,Basketball Court
1413,1.32044,103.886664,Chinese Restaurant,0,Food Court,Asian Restaurant,Chinese Restaurant,Grocery Store,Vegetarian / Vegan Restaurant,Noodle House,Fast Food Restaurant,Bistro,Bakery,Basketball Court
1414,1.32044,103.885875,Fast Food Restaurant,0,Food Court,Asian Restaurant,Chinese Restaurant,Grocery Store,Vegetarian / Vegan Restaurant,Noodle House,Fast Food Restaurant,Bistro,Bakery,Basketball Court
1415,1.32044,103.885547,Asian Restaurant,0,Food Court,Asian Restaurant,Chinese Restaurant,Grocery Store,Vegetarian / Vegan Restaurant,Noodle House,Fast Food Restaurant,Bistro,Bakery,Basketball Court


In [24]:
centre_merged.loc[centre_merged['Cluster Labels'] == 2, centre_merged.columns[[1] + list(range(5, centre_merged.shape[1]))]]

,Centre Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,1.28944,103.851548,Art Gallery,2,Japanese Restaurant,Yoga Studio,Nightclub,Italian Restaurant,Cocktail Bar,Hotel,Bar,Bistro,Shopping Mall,History Museum
8,1.28944,103.851691,French Restaurant,2,Japanese Restaurant,Yoga Studio,Nightclub,Italian Restaurant,Cocktail Bar,Hotel,Bar,Bistro,Shopping Mall,History Museum
9,1.28944,103.849381,Nightclub,2,Japanese Restaurant,Yoga Studio,Nightclub,Italian Restaurant,Cocktail Bar,Hotel,Bar,Bistro,Shopping Mall,History Museum
10,1.28944,103.850121,Shopping Mall,2,Japanese Restaurant,Yoga Studio,Nightclub,Italian Restaurant,Cocktail Bar,Hotel,Bar,Bistro,Shopping Mall,History Museum
11,1.28944,103.851731,Monument / Landmark,2,Japanese Restaurant,Yoga Studio,Nightclub,Italian Restaurant,Cocktail Bar,Hotel,Bar,Bistro,Shopping Mall,History Museum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348,1.28944,103.845552,Hotpot Restaurant,2,Japanese Restaurant,Yoga Studio,Nightclub,Italian Restaurant,Cocktail Bar,Hotel,Bar,Bistro,Shopping Mall,History Museum
1349,1.28944,103.853019,Lounge,2,Japanese Restaurant,Yoga Studio,Nightclub,Italian Restaurant,Cocktail Bar,Hotel,Bar,Bistro,Shopping Mall,History Museum
1350,1.28944,103.852207,Breakfast Spot,2,Japanese Restaurant,Yoga Studio,Nightclub,Italian Restaurant,Cocktail Bar,Hotel,Bar,Bistro,Shopping Mall,History Museum
1351,1.28944,103.852616,Restaurant,2,Japanese Restaurant,Yoga Studio,Nightclub,Italian Restaurant,Cocktail Bar,Hotel,Bar,Bistro,Shopping Mall,History Museum


In [25]:
centre_merged.loc[centre_merged['Cluster Labels'] == 3, centre_merged.columns[[1] + list(range(5, centre_merged.shape[1]))]]

,Centre Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
810,1.28073,103.786661,Chinese Restaurant,3,Bus Station,Grocery Store,Bar,Comfort Food Restaurant,Café,Metro Station,Chinese Restaurant,Indian Restaurant,Pizza Place,Convenience Store
811,1.28073,103.786852,Bar,3,Bus Station,Grocery Store,Bar,Comfort Food Restaurant,Café,Metro Station,Chinese Restaurant,Indian Restaurant,Pizza Place,Convenience Store
812,1.28073,103.787011,Café,3,Bus Station,Grocery Store,Bar,Comfort Food Restaurant,Café,Metro Station,Chinese Restaurant,Indian Restaurant,Pizza Place,Convenience Store
813,1.28073,103.781921,Metro Station,3,Bus Station,Grocery Store,Bar,Comfort Food Restaurant,Café,Metro Station,Chinese Restaurant,Indian Restaurant,Pizza Place,Convenience Store
814,1.28073,103.785504,Convenience Store,3,Bus Station,Grocery Store,Bar,Comfort Food Restaurant,Café,Metro Station,Chinese Restaurant,Indian Restaurant,Pizza Place,Convenience Store
815,1.28073,103.785740,Grocery Store,3,Bus Station,Grocery Store,Bar,Comfort Food Restaurant,Café,Metro Station,Chinese Restaurant,Indian Restaurant,Pizza Place,Convenience Store
816,1.28073,103.786736,Indian Restaurant,3,Bus Station,Grocery Store,Bar,Comfort Food Restaurant,Café,Metro Station,Chinese Restaurant,Indian Restaurant,Pizza Place,Convenience Store
817,1.28073,103.785865,Bar,3,Bus Station,Grocery Store,Bar,Comfort Food Restaurant,Café,Metro Station,Chinese Restaurant,Indian Restaurant,Pizza Place,Convenience Store
818,1.28073,103.784264,Bus Station,3,Bus Station,Grocery Store,Bar,Comfort Food Restaurant,Café,Metro Station,Chinese Restaurant,Indian Restaurant,Pizza Place,Convenience Store
819,1.28073,103.786669,Grocery Store,3,Bus Station,Grocery Store,Bar,Comfort Food Restaurant,Café,Metro Station,Chinese Restaurant,Indian Restaurant,Pizza Place,Convenience Store


In [26]:
centre_merged.loc[centre_merged['Cluster Labels'] == 4, centre_merged.columns[[1] + list(range(5, centre_merged.shape[1]))]]

,Centre Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1.32926,103.815565,Japanese Restaurant,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,1.32926,103.816648,Hotel,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
2,1.32926,103.815887,Japanese Restaurant,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
3,1.32926,103.813910,Modern European Restaurant,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
4,1.32926,103.815864,Lounge,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
5,1.32926,103.816660,Café,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
6,1.32926,103.817763,Park,4,Japanese Restaurant,Hotel,Lounge,Modern European Restaurant,Café,Park,French Restaurant,Food Truck,Food Court,Food & Drink Shop
